# Artificial Neural Network

### Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Importing the dataset

In [3]:
dataset = pd.read_excel('Peak_Incident_Pressure.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   Peak_Pressure      576 non-null    float64
 3   Peak_time          576 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 18.1 KB


In [4]:
y = dataset['Peak_time']
X = dataset.drop(['Peak_time','Peak_Pressure'], axis=1)
print(X.shape, y.shape)

(576, 2) (576,)


In [5]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning 1 - layers, neurons, activation function

In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ 2
    nodes = first_layer_nodes
    for i in range(1, 4):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [8]:
from tensorflow.keras.optimizers import Adam
def create_model(first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(first_layer_nodes, last_layer_nodes)
    for i in range(1, 4):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_83412\867054689.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [9]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(first_layer_nodes = [150, 200, 250],
                  last_layer_nodes = [80, 100, 120],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 1s 4ms/step - loss: 1.9005 - mae: 1.0851
Epoch 2/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5885 - mae: 0.6391
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 0.1823 - mae: 0.3439
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0826 - mae: 0.2261
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0331 - mae: 0.1386
Epoch 6/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0137 - mae: 0.0981
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0066 - mae: 0.0642
Epoch 8/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0027 - mae: 0.0386
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 0.0013 - mae: 0.0273
Epoch 10/500
9/9 [==============================] - 0s 3ms/step - loss: 8.3220e-04 - mae: 0.0217
Epoch 11/500
9/9 [==============================] - 0s 3ms/step - loss: 6.5

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [10]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=250, input_shape=(X_train.shape[1],), activation='leaky_relu'))
 model.add(Dense(units=185, activation='leaky_relu'))
 model.add(Dense(units=120, activation='leaky_relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [11]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [20,30,40]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/400
21/21 [==============================] - 1s 3ms/step - loss: 5.2025 - mae: 0.9158
Epoch 2/400
21/21 [==============================] - 0s 2ms/step - loss: 0.4632 - mae: 0.4341
Epoch 3/400
21/21 [==============================] - 0s 2ms/step - loss: 0.3845 - mae: 0.3821
Epoch 4/400
21/21 [==============================] - 0s 2ms/step - loss: 0.2548 - mae: 0.3180
Epoch 5/400
21/21 [==============================] - 0s 2ms/step - loss: 0.2237 - mae: 0.3453
Epoch 6/400
21/21 [==============================] - 0s 2ms/step - loss: 0.1263 - mae: 0.2492
Epoch 7/400
21/21 [==============================] - 0s 2ms/step - loss: 0.1511 - mae: 0.2564
Epoch 8/400
21/21 [==============================] - 0s 2ms/step - loss: 0.0332 - mae: 0.0879
Epoch 9/400
21/21 [==============================] - 0s 2ms/step - loss: 0.0249 - mae: 0.1353
Epoch 10/400
21/21 [==============================] - 0s 2ms/step - loss: 0.0536 - mae: 0.1689
Epoch 11/400
21/21 [==============================] - 0s 2m